# Demonstration notebook for processing raw RINEX data
In this Notebook, we process some example RINEX files to demonstrate gnssvod.

In [1]:
import gnssvod as gv

## gv.preprocess()
The main pre-processing function is preprocess(). This function  will do several things
- It will read RINEX observation files as pandas data frames
- It can aggregate the raw data to a lower temporal rate if specified.
- It will by default download orbit and clock files for the corresponding days from the GSSC ESA server
- From the orbit and clock files, it will calculate azimuth and elevation for each measurement
- It can save each processed file as a netcdf file in the outputdir folder or return the results as a dictionary

### specifying input files
The function exclusively reads RINEX observation files. Such files typically end with the extension '.yyO' where yy is the last two digit of the year. The function can be used to process a single file, a group of files, or several groups of files corresponding to several receivers, as shown in the examples below. All of this is done by specifying a pattern as the first argument to the function.

### specifying output destinations
Results are saved to a NetCDF file when an output directory is specified and/or returned as a dictionary when "outputresult=True" is passed.

Let's read a single file using the example data to begin with

In [6]:
pattern = {'MACROCOSM-5':'data_pr/MACROCOSM-5_raw_202309251326.23O'}
result = gv.preprocess(pattern,outputresult=True)
#here got the same error on integer module zero but I am able to process data below -CVR

data_pr/MACROCOSM-5_raw_202309251326.23O exists | Reading...
Observation file  data_pr/MACROCOSM-5_raw_202309251326.23O  is read in 6.80 seconds.
Processing 392681 individual observations
Calculating Azimuth and Elevation
GFZ0MGXRAP_20232680000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20232680000_01D_05M_ORB.SP3 file is read in 0.31 seconds
GFZ0MGXRAP_20232680000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20232680000_01D_30S_CLK.CLK file is read in 1.36 seconds


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")


ZeroDivisionError: integer modulo by zero

The default logs should indicate how many observations were read in the file. If this is the first time you run the script, it also shows some orbit files were downloaded.

If you process very recent data (less than 3 days old), it could be that the orbit and clock files are not available on the ESA server yet and there would then be an error.

The result returned by the function is a dictionary providing lists of Observation objects.

In [7]:
result

{'MACROCOSM-1': [<gnssvod.io.io.Observation at 0x140caf1d0>]}

Since we processed one file, there is only one Observation object in the list. Let us access this first and unique item.

In [8]:
obs = result['MACROCOSM-5'][0]
obs

KeyError: 'MACROCOSM-5'

Observation objects are custom classes introduced in the `gnsspy` package by Mustafa Serkan Işık and Volkan Özbey. A significant number of base functions in `gnssvod` are based on gnsspy.

Observation objects contain the following properties
- obs.filename          = the name of the source file
- obs.epoch             = a datetime indicate the day at the start of the record
- obs.observation       = a pandas data frame containing all measurements
- obs.approx_position   = the approximate receiver position as provided in the RINEX file [X,Y,Z]
- obs.receiver_type     = the receiver type if provided in the RINEX file
- obs.antenna_type      = the antenna type if provided in the RINEX file
- obs.interval          = the measurement frequency in seconds
- obs.receiver_clock    = the receiver clock if provided in the RINEX file
- obs.version           = the version of the RINEX file
- obs.observation_types = the observation types reported as columns in obs.observation

Let's just look at the data..

In [5]:
obs.observation

C1C  C1X  C2C           C2I           C2X  \
Epoch               SV                                                        
2023-09-25 13:32:16 G05  1.978206e+07  NaN  NaN           NaN  1.978206e+07   
                    G11  1.987883e+07  NaN  NaN           NaN  1.987884e+07   
                    G13  1.811845e+07  NaN  NaN           NaN           NaN   
                    G15  1.922018e+07  NaN  NaN           NaN  1.922019e+07   
                    G20  2.055847e+07  NaN  NaN           NaN           NaN   
...                               ...  ...  ...           ...           ...   
2023-09-25 14:24:08 C20           NaN  NaN  NaN  2.178272e+07           NaN   
                    C27           NaN  NaN  NaN  2.133223e+07           NaN   
                    C29           NaN  NaN  NaN  2.376635e+07           NaN   
                    C30           NaN  NaN  NaN  2.041070e+07           NaN   
                    C32           NaN  NaN  NaN  2.175568e+07           NaN   

                         C7I  C7X       D1C  D1X  D2C  ...  L7X   S1C  S1X  \
Epoch               SV                                 ...                   
2023-09-25 13:32:16 G05  NaN  NaN -1330.471  NaN  NaN  ...  NaN  47.0  NaN   
                    G11  NaN  NaN -1227.567  NaN  NaN  ...  NaN  43.0  NaN   
                    G13  NaN  NaN  -458.842  NaN  NaN  ...  NaN  46.0  NaN   
                    G15  NaN  NaN  1511.608  NaN  NaN  ...  NaN  44.0  NaN   
                    G20  NaN  NaN -1738.815  NaN  NaN  ...  NaN  44.0  NaN   
...                      ...  ...       ...  ...  ...  ...  ...   ...  ...   
2023-09-25 14:24:08 C20  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C27  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C29  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C30  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C32  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   

                         S2C   S2I   S2X  S7I  S7X     Azimuth  Elevation  
Epoch               SV                                                     
2023-09-25 13:32:16 G05  NaN   NaN  39.0  NaN  NaN   -5.700510  44.589544  
                    G11  NaN   NaN  38.0  NaN  NaN   80.463045  46.268726  
                    G13  NaN   NaN   NaN  NaN  NaN  105.015143  79.936581  
                    G15  NaN   NaN  36.0  NaN  NaN -131.300048  53.269387  
                    G20  NaN   NaN   NaN  NaN  NaN   35.345305  33.793078  
...                      ...   ...   ...  ...  ...         ...        ...  
2023-09-25 14:24:08 C20  NaN  47.0   NaN  NaN  NaN  -75.772562  41.205882  
                    C27  NaN  45.0   NaN  NaN  NaN  159.124746  48.408466  
                    C29  NaN  43.0   NaN  NaN  NaN  -39.814478  18.330130  
                    C30  NaN  46.0   NaN  NaN  NaN  -74.370507  70.158474  
                    C32  NaN  47.0   NaN  NaN  NaN    7.447608  45.350861  

[78464 rows x 30 columns]

The pandas data frame has a MultIndex that contains both Epoch and SV as indices. The columns correspond to:
- C# = Pseudorange from the receiver to the satellite, in meters
- L# = Carrier phase, in cycles
- D# = Doppler, in Hz
- S# = Carrier to noise density C/N$_0$, in dB (receiver-dependent)

And the numbers (S1, S2, etc. ) indicate the corresponding GNSS frequency

The azimuth and elevation of the satellite with respect to the receiver are expressed in degrees. Computation speed for the azimuth and elevation can vary according to your hardware. Most of the time is spent interpolating the orbit parameters to the time stamps of each measurement. This is why it is sometimes useful to aggregate high frequency data (here one measurement per second) to for instance one measurement each 15 seconds.

### resampling

We can pass "interval='15S'" to resample the data during the preprocessing. The returned data will be smaller and the calculation of the azimuths and elevations (reported as "SP3 interpolation") will be faster.

In [9]:
pattern = {'MACROCOSM-5':'data_pr/MACROCOSM-5_raw_202309251326.23O'}
result = gv.preprocess(pattern,interval='15S',outputresult=True)
# and show data frame
result['MACROCOSM-5'][0].observation

data_pr/MACROCOSM-5_raw_202309251326.23O exists | Reading...
Observation file  data_pr/MACROCOSM-5_raw_202309251326.23O  is read in 7.03 seconds.
Processing 392681 individual observations


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:200: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  obs.observation = obs.observation[subset].groupby([pd.Grouper(freq=interval, level='Epoch'),pd.Grouper(level='SV')]).mean()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:204: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  obs.interval = pd.Timedelta(interval).seconds


Calculating Azimuth and Elevation
GFZ0MGXRAP_20232680000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20232680000_01D_05M_ORB.SP3 file is read in 0.55 seconds
GFZ0MGXRAP_20232680000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20232680000_01D_30S_CLK.CLK file is read in 1.38 seconds


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a futur

SP3 interpolation is done in 0.89 seconds


C1C           C1X           C2C  \
Epoch               SV                                              
2023-09-25 13:26:15 C20           NaN           NaN           NaN   
                    C27           NaN           NaN           NaN   
                    C30           NaN           NaN           NaN   
                    C32           NaN           NaN           NaN   
                    E04           NaN  2.448772e+07           NaN   
...                               ...           ...           ...   
2023-09-25 14:18:30 R02  2.466551e+07           NaN           NaN   
                    R08  2.341971e+07           NaN  2.341972e+07   
                    R22  2.296789e+07           NaN  2.296790e+07   
                    R23  2.098165e+07           NaN           NaN   
                    R24  2.367392e+07           NaN  2.367393e+07   

                                  C2I  C2X  C7I           C7X          D1C  \
Epoch               SV                                                       
2023-09-25 13:26:15 C20  2.493057e+07  NaN  NaN           NaN          NaN   
                    C27  2.283638e+07  NaN  NaN           NaN          NaN   
                    C30  2.379760e+07  NaN  NaN           NaN          NaN   
                    C32  2.345880e+07  NaN  NaN           NaN          NaN   
                    E04           NaN  NaN  NaN  2.448773e+07          NaN   
...                               ...  ...  ...           ...          ...   
2023-09-25 14:18:30 R02           NaN  NaN  NaN           NaN  1054.896000   
                    R08           NaN  NaN  NaN           NaN -3939.939634   
                    R22           NaN  NaN  NaN           NaN -4113.566659   
                    R23           NaN  NaN  NaN           NaN  -331.972439   
                    R24           NaN  NaN  NaN           NaN  2745.659390   

                                D1X          D2C  ...           L7X  \
Epoch               SV                            ...                 
2023-09-25 13:26:15 C20         NaN          NaN  ...           NaN   
                    C27         NaN          NaN  ...           NaN   
                    C30         NaN          NaN  ...           NaN   
                    C32         NaN          NaN  ...           NaN   
                    E04 -585.227333          NaN  ...  9.860197e+07   
...                             ...          ...  ...           ...   
2023-09-25 14:18:30 R02         NaN          NaN  ...           NaN   
                    R08         NaN -3064.450488  ...           NaN   
                    R22         NaN -3199.421732  ...           NaN   
                    R23         NaN          NaN  ...           NaN   
                    R24         NaN  2135.532927  ...           NaN   

                               S1C        S1X        S2C        S2I  S2X  S7I  \
Epoch               SV                                                          
2023-09-25 13:26:15 C20        NaN        NaN        NaN  42.789474  NaN  NaN   
                    C27        NaN        NaN        NaN  47.929825  NaN  NaN   
                    C30        NaN        NaN        NaN  46.385965  NaN  NaN   
                    C32        NaN        NaN        NaN  46.421053  NaN  NaN   
                    E04        NaN  45.807018        NaN        NaN  NaN  NaN   
...                            ...        ...        ...        ...  ...  ...   
2023-09-25 14:18:30 R02  25.560976        NaN        NaN        NaN  NaN  NaN   
                    R08  42.707317        NaN  32.268293        NaN  NaN  NaN   
                    R22  47.707317        NaN  43.000000        NaN  NaN  NaN   
                    R23  45.000000        NaN        NaN        NaN  NaN  NaN   
                    R24  43.390244        NaN  34.365854        NaN  NaN  NaN   

                          S7X     Azimuth  Elevation  
Epoch               SV                                
2023-09-25 13:26:15 C20   NaN -10

Orbit and clock files are not downloaded again if they already exist. There are now less rows in the data frame.

## Batch processing
We now use the preprocessing function to process many files and save the outputs as NetCDF files (instead of returning as objects). If we were to process several hundreds of files, the system would likely not have sufficient memory to hold all of the outputs, so it makes sense to processed data as a NetCDF file.

### Specifying several groups of files
Instead of specifying just one file, we use the dictionary to specify a pattern. All files matching the pattern will be processed. We can process several groups files by specifying different matching patterns (see below).

### Specifying where to save data
Same as for specifying the inputs, we use a dictionary to indicate where to save data. The function will create the destination folder if it does not exist.

### Specifying a list of variables to save
For calculating GNSS-VOD, we only need the "S" variables. We can reduce the size of the saved NetCDF files by discarding the other variables, this is done with the 'keepvars' argument, which will only keep the variables present in the passed list. This argument supports UNIX-style pattern matching (e.g. 'S*' will match all variables starting with 'S')

### Compression
Unless `compress=False` is passed as argument, `gv.preprocess()` will compress all S* variables, as well as Azimuth and Elevation when saving to NetCDF. These variables are encoded as Int16 with a scale factor of 0.1. The decoding is automatically applied when reading the data with xarray.

In [11]:
# use gnssvod to batch process the observation RINEX files 
# (files with extension .yyO for each station)
# pattern = {'choice_of_name_for_station1':'pattern to match (UNIX-style)',
#            'choice_of_name_for_station2':'pattern to match (UNIX-style)',
#             ...}
#
pattern = {'MACROCOSM-5':'data_pr/MACROCOSM-5*.23O',}
outputdir = {'MACROCOSM-5':'data_pr/nc/',}
# what variables should be kept
keepvars = ['S1C', 'S1X', 'S2C', 'S2X']

gv.preprocess(pattern,interval='15S',keepvars=keepvars,outputdir=outputdir)

data_pr/MACROCOSM-5_raw_202309251326.23O exists | Reading...
Observation file  data_pr/MACROCOSM-5_raw_202309251326.23O  is read in 6.90 seconds.
Processing 392681 individual observations
Calculating Azimuth and Elevation
GFZ0MGXRAP_20232680000_01D_05M_ORB.SP3 exists | Reading...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:200: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  obs.observation = obs.observation[subset].groupby([pd.Grouper(freq=interval, level='Epoch'),pd.Grouper(level='SV')]).mean()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:204: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  obs.interval = pd.Timedelta(interval).seconds


GFZ0MGXRAP_20232680000_01D_05M_ORB.SP3 file is read in 0.33 seconds
GFZ0MGXRAP_20232680000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20232680000_01D_30S_CLK.CLK file is read in 1.39 seconds


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a futur

SP3 interpolation is done in 0.87 seconds
Saved 5475 individual observations in MACROCOSM-5_raw_202309251326.nc
data_pr/MACROCOSM-5_raw_202309151848.23O exists | Reading...
Observation file  data_pr/MACROCOSM-5_raw_202309151848.23O  is read in 0.13 seconds.
Processing 7675 individual observations
Calculating Azimuth and Elevation
This file does not exist: GFZ0MGXRAP_20232580000_01D_05M_ORB.SP3
Downloading: GFZ0MGXRAP_20232580000_01D_05M_ORB.SP3.gz

...

 | Download completed for GFZ0MGXRAP_20232580000_01D_05M_ORB.SP3.gz
 | Requested file GFZ0MGXRAP_20232580000_01D_05M_ORB.SP3.gz cannot be not found!
GFZ0MGXRAP_20232580000_01D_05M_ORB.SP3 file is read in 2.50 seconds
This file does not exist: GFZ0MGXRAP_20232580000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20232580000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20232580000_01D_30S_CLK.CLK.gz: 4.34MB [00:01, 2.36MB/s]             


 | Download completed for GFZ0MGXRAP_20232580000_01D_30S_CLK.CLK.gz
GFZ0MGXRAP_20232580000_01D_30S_CLK.CLK file is read in 1.34 seconds


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a futur

SP3 interpolation is done in 0.88 seconds
Saved 113 individual observations in MACROCOSM-5_raw_202309151848.nc
